In [1]:
import os
os.chdir('/app/Pogona_realtime/Arena')

In [2]:
from Prediction import dataset
from Prediction import kalman_predict
from Prediction import visualize
from Prediction import train_eval

import imp
import numpy as np
import matplotlib.pyplot as plt
from time import time
from ax.service.managed_loop import optimize
import torch

In [14]:
imp.reload(dataset)
imp.reload(visualize)
imp.reload(train_eval)

<module 'Prediction.train_eval' from '/app/Pogona_realtime/Arena/Prediction/train_eval.py'>

In [3]:
all_df = dataset.collect_data(data_sources={'detections': True, 'timestamps': True, 'dlc': False, 'touches': True})

46 trials loaded


In [4]:
trial_names = list(all_df.index.unique())
trial_names

['feeding_20200830-115520_None',
 'feeding_20200830-120813_None',
 'feeding_4_20200830-121143_None',
 'feeding_4_20200830-122515_None',
 'feeding_4_20200830-125531_None',
 'feeding_4_20200830-125905_None',
 'feeding_4_screen_20200830-130446_None',
 'feeding_5_20200830-131348_None',
 'line_20200803T081429_trial1',
 'line_20200803T081735_trial1',
 'line_20200803T082002_trial1',
 'fast_line_20200803T082414_trial1',
 'fast_line_20200803T082724_trial1',
 'circle_20200803T084227_trial1',
 'circle_20200803T084227_trial2',
 'circle_20200803T084529_trial1',
 'circle_20200803T084529_trial2',
 'fast_cockroach_20200803T085738_trial1',
 'circle_20200803T090234_trial1',
 'circle_20200803T090615_trial1',
 'circle_20200803T091639_trial1',
 'circle_20200803T092248_trial1',
 'circle_20200803T093051_trial1',
 'line_20200803T094928_trial1',
 'line_20200803T095401_trial1',
 'line_20200803T101641_trial1',
 'line_20200803T101641_trial2',
 'circle_20200803T113121_trial1',
 'line_20200803T114942_trial1',
 'cir

In [22]:
trial = trial_names[8]
homography = dataset.homography_for_trial(trial)
correction_fn = visualize.get_correction_fn(homography,1920)
vid_path = dataset.get_trial_video_path(trial)
bboxes = all_df.loc[trial][['x1', 'y1', 'x2', 'y2']].values

In [26]:
# no movement: 0s-4s, 32s-40s
still_bboxes1 = bboxes[0:4*60]
still_bboxes2 = bboxes[32*60:40*60]
still_bboxes1 = still_bboxes1[~np.any(np.isnan(still_bboxes1), axis=1)]
still_bboxes2 = still_bboxes2[~np.any(np.isnan(still_bboxes2), axis=1)]

In [36]:
cov1 = np.cov(still_bboxes1, rowvar=False)
cov2 = np.cov(still_bboxes2, rowvar=False)

In [37]:
cov1

array([[10.04448372,  5.28191206, -4.32083342, -6.1339851 ],
       [ 5.28191206,  6.54297075, -2.0591427 , -4.13508616],
       [-4.32083342, -2.0591427 ,  7.84421984,  4.1640904 ],
       [-6.1339851 , -4.13508616,  4.1640904 ,  9.02270658]])

In [38]:
cov2

array([[ 3.01217452,  0.32374946,  0.27029129,  0.10563435],
       [ 0.32374946,  2.6565541 , -0.18394073, -0.30784146],
       [ 0.27029129, -0.18394073,  4.41411549, -0.92239415],
       [ 0.10563435, -0.30784146, -0.92239415,  5.07934416]])

In [55]:
def filter_bboxes(bboxes, R, q_var):
    traj_inited = False
    filtered_bboxes = np.empty(bboxes.shape)
    filtered_bboxes[:] = np.nan

    for i in range(bboxes.shape[0]):
        if not np.any(np.isnan(bboxes[i])):
            if traj_inited:
                kf.update(bboxes[i])
            else:
                traj_inited = True
                init_x = np.zeros(8, np.float)
                init_x[0::2] = bboxes[i]
                kf = kalman_predict.create_kalman_filter(input_dim=4, init_x=init_x, r_var=50.0, q_var=q_var)
                kf.R = R
    
        if traj_inited:
            kf.predict()
            filtered_bboxes[i] = kf.x[0::2]
    
    return filtered_bboxes

In [77]:
out_path = '../labelled/2309_test_kalman_filter.mp4'
filtered_bboxes = filter_bboxes(bboxes, R=cov2, q_var=1.0)
visualize.process_video(video_path=vid_path,
                        output_path=out_path,
                        process_fns=[ #visualize.offline_bbox_visualizer(bboxes, window_size=1),
                                     visualize.offline_bbox_visualizer(filtered_bboxes, color=(255,0,0), window_size=1)],
                        correction_fn = correction_fn,
#                        num_frames=500,
                        resize_to_width=960)

100%|██████████| 5081/5081 [02:17<00:00, 36.98it/s]


In [78]:
filtered_cov = np.cov(filtered_bboxes[32*60:40*60], rowvar=False)
filtered_cov

array([[ 3.35038854,  0.15467331,  0.31965062, -0.03229599],
       [ 0.15467331,  3.16603278, -0.01436603, -0.25792195],
       [ 0.31965062, -0.01436603,  4.5760921 , -0.51729725],
       [-0.03229599, -0.25792195, -0.51729725,  5.16254736]])

## Look for Q using Ax and filtering

In [59]:
parameters=[
    {"name": "q_var", "type": "range", "bounds": [0.0, 1.0]},
    ]
all_bboxes = all_df[['x1', 'y1', 'x2', 'y2']].values
tensor_bboxes = torch.tensor(all_bboxes).to('cuda')

In [73]:
def eval_kalman(params):
    filtered = filter_bboxes(all_bboxes, R=cov2, q_var=params["q_var"])
    nan_mask = np.any(np.isnan(all_bboxes), axis=1)
    tensor_filtered = torch.tensor(filtered).to('cuda')
    ade = train_eval.calc_ADE(tensor_filtered[~nan_mask], tensor_bboxes[~nan_mask])
    print(f"params: {params} ADE:{ade.item()}")
    return ade.item()

In [74]:
best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=eval_kalman,
    objective_name='ADE',
    total_trials=100, 
    minimize=True)

[INFO 09-23 17:18:21] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due to  model-fitting.
[INFO 09-23 17:18:21] ax.service.managed_loop: Started full optimization with 100 steps.
[INFO 09-23 17:18:21] ax.service.managed_loop: Running optimization trial 1...
[INFO 09-23 17:18:34] ax.service.managed_loop: Running optimization trial 2...


params: {'q_var': 0.036391002126038074} ADE:4.777022232638083


[INFO 09-23 17:18:47] ax.service.managed_loop: Running optimization trial 3...


params: {'q_var': 0.7990192454308271} ADE:3.0774003273565387


[INFO 09-23 17:18:59] ax.service.managed_loop: Running optimization trial 4...


...

params: {'q_var': 0.558722703717649} ADE:3.208517844456468


/home/user/miniconda/envs/py37/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:46: NumericalWarning:

A not p.d., added jitter of 1.0e-08 to the diagonal

/home/user/miniconda/envs/py37/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:46: NumericalWarning:

A not p.d., added jitter of 1.0e-08 to the diagonal

[INFO 09-23 17:40:07] ax.service.managed_loop: Running optimization trial 100...


params: {'q_var': 1.0} ADE:3.009111633963276


/home/user/miniconda/envs/py37/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:46: NumericalWarning:

A not p.d., added jitter of 1.0e-08 to the diagonal

/home/user/miniconda/envs/py37/lib/python3.7/site-packages/gpytorch/utils/cholesky.py:46: NumericalWarning:

A not p.d., added jitter of 1.0e-08 to the diagonal



params: {'q_var': 1.0} ADE:3.009111633963276


In [75]:
best_parameters

{'q_var': 1.0}

## Search for q_var and r_var for prediction

In [21]:
parameters=[
    {"name": "q_var", "type": "range", "bounds": [0.0, 100.0]},
    {"name": "r_var", "type": "range", "bounds": [0.0, 200.0]},
    ]

In [22]:
def eval_kalman_predictor(params):
    total_ADE = 0
    start_time = time()
    
    for trial in trial_names:        
        trial_bboxes = all_df.loc[trial][['x1', 'y1', 'x2', 'y2']].values        
        predictor = kalman_predict.KalmanPredictor(20, 1, params["q_var"], params["r_var"])
        results, forecasts = train_eval.eval_trajectory_predictor(predictor, trial_bboxes, show_progress=False)
        total_ADE += results["avg ADE"]

        print(".", end="")
    mean_ADE = total_ADE / len(trial_names)
    print(f"params: {params} ADE:{mean_ADE} duration: {time() - start_time:.2f}")
    
    return mean_ADE

In [23]:
best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=eval_kalman_predictor,
    objective_name='ADE',
    total_trials=100, 
    minimize=True)

[INFO 09-23 21:12:38] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due to  model-fitting.
[INFO 09-23 21:12:38] ax.service.managed_loop: Started full optimization with 100 steps.
[INFO 09-23 21:12:38] ax.service.managed_loop: Running optimization trial 1...


.............................................

[INFO 09-23 21:13:43] ax.service.managed_loop: Running optimization trial 2...


.params: {'q_var': 57.74761764332652, 'r_var': 148.13252426683903} ADE:25.462888330917142 duration: 65.14
.............................................

[INFO 09-23 21:14:50] ax.service.managed_loop: Running optimization trial 3...


...

.params: {'q_var': 27.38439155141072, 'r_var': 33.133552991001245} ADE:28.664449443240507 duration: 67.02
.............................................

[INFO 09-23 23:00:35] ax.service.managed_loop: Running optimization trial 99...


.params: {'q_var': 5.370325791258682, 'r_var': 120.38665794106605} ADE:20.309112430927975 duration: 67.55
.............................................

[INFO 09-23 23:01:42] ax.service.managed_loop: Running optimization trial 100...


.params: {'q_var': 75.89974326037225, 'r_var': 106.02438063668413} ADE:27.977663721500793 duration: 66.95
..............................................params: {'q_var': 83.59791134235954, 'r_var': 191.93412838293062} ADE:25.877175436377136 duration: 66.79


### Best parameters for constant acceleration model

- {'q_var': 5.232464612645584, 'r_var': 76.11997188671926} ADE:77.80251293150333
- {'q_var': 8.305751509936852e-11, 'r_var': 6.104439686140315} ADE:56.59084980428664
- {'q_var': 11.243166495979125, 'r_var': 100.0} ADE:87.78338522412209
- {'q_var': 6.60728768670246, 'r_var': 68.14068332725518} ADE:84.5807759587622
- {'q_var': 3.179927344778319e-08, 'r_var': 99.99999991206423} ADE:47.3384215434221
- {'q_var': 4.584311371533344, 'r_var': 99.99999999999997} ADE:70.82873570482703

### Best parameters for constant velocity model

- {'q_var': 5.370325791258682, 'r_var': 120.38665794106605} ADE:20.309112430927975
- {'q_var': 17.93990045743253, 'r_var': 134.50567199523354} ADE:22.291919662508487
- {'q_var': 31.548048307438687, 'r_var': 199.99999999999997} ADE:22.69878701033521
- {'q_var': 17.266130167990923, 'r_var': 162.3100133612752} ADE:21.791229502949385
- {'q_var': 2.1451830012770112, 'r_var': 119.49832681948244} ADE:19.396253677900415
- {'q_var': 10.68977698973989, 'r_var': 104.58652082983697} ADE:21.706828026167155
- {'q_var': 20.018173460688566, 'r_var': 180.35298309143698} ADE:21.881616195822115